In [1]:
from src.utils import *

In [2]:
# import wav data to keras dataset
import numpy as np
from keras.utils import audio_dataset_from_directory

# Load audio dataset from directory
train_dataset, validation_dataset = audio_dataset_from_directory(
    directory='data/path/train',
    validation_split=0.2,
    subset="both",
    labels="inferred",
    label_mode="categorical",
    seed=1337,
    batch_size=64,
    output_sequence_length=16000, 
)


2024-04-22 15:14:32.131699: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-22 15:14:32.460301: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-22 15:14:33.241035: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Found 64721 files belonging to 30 classes.
Using 51777 files for training.
Using 12944 files for validation.


2024-04-22 15:14:57.681618: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-22 15:14:57.773516: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-22 15:14:57.773551: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-22 15:14:57.775807: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-22 15:14:57.775838: I external/local_xla/xla/stream_executor

In [3]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))
from tensorflow.keras.optimizers import Adam
from transformers import AutoFeatureExtractor, TFWav2Vec2ForSequenceClassification

# Load pre-trained Wav2Vec2 model and processor using AutoFeatureExtractor
model_name = "facebook/wav2vec2-base-960h"
tokenizer = AutoFeatureExtractor.from_pretrained(model_name)

# Load pre-trained Wav2Vec2 model
model = TFWav2Vec2ForSequenceClassification.from_pretrained(model_name)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]



TFWav2Vec2ForSequenceClassification has backpropagation operations that are NOT supported on CPU. If you wish to train/fine-tune this model, you need a GPU or a TPU
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFWav2Vec2ForSequenceClassification: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing TFWav2Vec2ForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFWav2Vec2ForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFWav2Vec2ForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['wav2vec2.masked_spec_embed', 'pr

In [4]:
import tensorflow as tf
from transformers import AutoFeatureExtractor
from tqdm.notebook import tqdm

# Load Wav2Vec tokenizer
model_name = "facebook/wav2vec2-base-960h"
tokenizer = AutoFeatureExtractor.from_pretrained(model_name)
print("test")

# Define a function to tokenize audio data
def tokenize_dataset(dataset):
    tokenized_batches = []
    for batch in tqdm(dataset, total=len(dataset)):
        
        audio_batch, labels = batch
        # print(audio_batch)
        inputs = tokenizer(audio_batch, return_tensors="tf", padding=True, verbose=False, sampling_rate=16000)
        tokenized_batches.append((inputs.input_values, labels))
    return tokenized_batches

# Tokenize the train dataset
tokenized_train_dataset = tokenize_dataset(train_dataset)

# Tokenize the validation dataset
tokenized_validation_dataset = tokenize_dataset(validation_dataset)


test


  0%|          | 0/810 [00:00<?, ?it/s]

2024-04-22 15:26:32.312595: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


  0%|          | 0/203 [00:00<?, ?it/s]

2024-04-22 15:29:27.108169: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


import tensorflow as tf

# Convert tokenized batches into a generator function
def batch_generator(tokenized_batches):
    for input_values, labels in tokenized_batches:
        yield input_values, labels

# Create Keras batch datasets for train and validation datasets
keras_train_dataset = tf.data.Dataset.from_generator(
    generator=lambda: batch_generator(tokenized_train_dataset),
    output_signature=(
        tf.TensorSpec(shape=(1, 32, 16000, 1), dtype=tf.float32),  # Input values
        tf.TensorSpec(shape=(32, 30), dtype=tf.int32)           # Labels
    )
)

keras_validation_dataset = tf.data.Dataset.from_generator(
    generator=lambda: batch_generator(tokenized_validation_dataset),
    output_signature=(
        tf.TensorSpec(shape=(1, 32, 16000, 1), dtype=tf.float32),  # Input values
        tf.TensorSpec(shape=(32, 30), dtype=tf.int32)           # Labels
    )
)



In [5]:
X_train = tf.concat([i[0][0] for i in tokenized_train_dataset], axis=0)
X_val = tf.concat([i[0][0] for i in tokenized_validation_dataset], axis=0)

In [6]:
y_train = tf.concat([i[1] for i in tokenized_train_dataset], axis=0)
y_val = tf.concat([i[1] for i in tokenized_validation_dataset], axis=0)

In [ ]:
import gc
del tokenized_train_dataset
del tokenized_validation_dataset
gc.collect()

In [7]:
X_train = X_train.numpy()
X_val = X_val.numpy()

In [8]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1])
X_val = X_val.reshape(X_val.shape[0], X_val.shape[1])

In [9]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Dropout, Dense
from tensorflow.keras.layers import Bidirectional, LSTM, TimeDistributed


def get_SR_Model(num_classes: int):
    X_input = Input(shape=(16000, 1))
    X = Conv1D(filters=256,kernel_size=15,strides=4)(X_input)
    X = BatchNormalization()(X)
    X = Dropout(0.2)(X)
    X = Conv1D(filters=512,kernel_size=15,strides=4)(X_input)
    X = BatchNormalization()(X)
    X = Dropout(0.2)(X)
    X = LSTM(units=512, return_sequences=True)(X)
    X = LSTM(units=512, return_sequences=False)(X)
    X = Dense(num_classes, activation='softmax')(X)
    return Model(inputs=[X_input], outputs=[X])

model = get_SR_Model(2)



In [10]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from transformers import TFWav2Vec2Model
from tensorflow.keras import Model, layers


class TFWav2Vec2Layer(layers.Layer):
    def __init__(self, **kwargs):
        super(TFWav2Vec2Layer, self).__init__(**kwargs)
        self.wav2vec_model = TFWav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")
        self.wav2vec_model.trainable = False

    def call(self, inputs, **kwargs):
        return self.wav2vec_model(inputs)["last_hidden_state"]


def get_wav2vec_classifier(num_classes: int):
    # Define input layer for tokenized input
    tokenized_input = layers.Input(shape=(None,), dtype=tf.int32)

    # Pass input through custom Wav2Vec2 layer
    hidden_states = TFWav2Vec2Layer()(tokenized_input)

    # Add classification layers
    x = layers.Dense(512, activation='relu')(hidden_states[:, 0, :])  # Take the first token's representation
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(num_classes, activation='softmax')(x)

    # Define model
    model = Model(inputs=tokenized_input, outputs=x)

    return model


# Example usage:
model = get_wav2vec_classifier(num_classes=30)



TFWav2Vec2Model has backpropagation operations that are NOT supported on CPU. If you wish to train/fine-tune this model, you need a GPU or a TPU
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFWav2Vec2Model: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing TFWav2Vec2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFWav2Vec2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFWav2Vec2Model were not initialized from the PyTorch model and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inf

In [11]:
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics

model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                loss=losses.CategoricalCrossentropy(),
                metrics=[metrics.CategoricalAccuracy()])

In [12]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=64)

Epoch 1/100


I0000 00:00:1713792588.711016    3431 service.cc:145] XLA service 0x7f1ae0010660 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1713792588.711069    3431 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 4090 Laptop GPU, Compute Capability 8.9
2024-04-22 15:29:49.138951: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
W0000 00:00:1713792589.306986    3431 assert_op.cc:38] Ignoring Assert operator functional_3_1/tf_wav2_vec2_layer_1/tf_wav2_vec2_model/wav2vec2/encoder/layers.0/attention/assert_equal_1/Assert/Assert
W0000 00:00:1713792589.722730    3431 assert_op.cc:38] Ignoring Assert operator functional_3_1/tf_wav2_vec2_layer_1/tf_wav2_vec2_model/wav2vec2/encoder/layers.0/attention/assert_equal_3/Assert/Assert
W0000 00:00:1713792589.722934    3431 assert_op.cc:38] Ignoring Assert operator functional_3_1/tf_wav2_v

  3/810 ━━━━━━━━━━━━━━━━━━━━ 47s 59ms/step - categorical_accuracy: 0.0382 - loss: 3.4199 

I0000 00:00:1713792599.783438    3431 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


809/810 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - categorical_accuracy: 0.0872 - loss: 3.2731

I0000 00:00:1713792651.554661   10425 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_6', 12 bytes spill stores, 12 bytes spill loads



810/810 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - categorical_accuracy: 0.0872 - loss: 3.2730

I0000 00:00:1713792672.287529   10741 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_539', 4 bytes spill stores, 4 bytes spill loads



810/810 ━━━━━━━━━━━━━━━━━━━━ 96s 95ms/step - categorical_accuracy: 0.0873 - loss: 3.2728 - val_categorical_accuracy: 0.2059 - val_loss: 2.8883
Epoch 2/100
810/810 ━━━━━━━━━━━━━━━━━━━━ 60s 74ms/step - categorical_accuracy: 0.2005 - loss: 2.8546 - val_categorical_accuracy: 0.2365 - val_loss: 2.7212
Epoch 3/100
810/810 ━━━━━━━━━━━━━━━━━━━━ 60s 74ms/step - categorical_accuracy: 0.2356 - loss: 2.6971 - val_categorical_accuracy: 0.2614 - val_loss: 2.6233
Epoch 4/100
810/810 ━━━━━━━━━━━━━━━━━━━━ 82s 73ms/step - categorical_accuracy: 0.2578 - loss: 2.6071 - val_categorical_accuracy: 0.2834 - val_loss: 2.5263
Epoch 5/100
810/810 ━━━━━━━━━━━━━━━━━━━━ 59s 73ms/step - categorical_accuracy: 0.2820 - loss: 2.5339 - val_categorical_accuracy: 0.2841 - val_loss: 2.5250
Epoch 6/100
810/810 ━━━━━━━━━━━━━━━━━━━━ 60s 74ms/step - categorical_accuracy: 0.2775 - loss: 2.5336 - val_categorical_accuracy: 0.2982 - val_loss: 2.4648
Epoch 7/100
810/810 ━━━━━━━━━━━━━━━━━━━━ 60s 74ms/step - categorical_accuracy: 0.2